In [94]:
import pandas as pd

In [95]:
df = pd.read_csv('../../../data/extracted/cust_wallet_detail_validation_without_truth.csv')
train_send_df = pd.read_csv('../../../data/extracted/cust_coupon_detail_send_train.csv')
raw_train_use_df = pd.read_csv('../../../data/extracted/cust_coupon_detail_used_train.csv')
# 将raw_train_user_df按照membercode分组，然后按照8:2的比例划分为train_use_df和valid_use_df
# 先获取所有membercode
all_membercodes = raw_train_use_df['membercode'].unique()
# 随机抽取80%的membercode作为训练，其余20%作为验证
import numpy as np
np.random.seed(42)
train_membercodes = np.random.choice(all_membercodes, size=int(0.8 * len(all_membercodes)), replace=False)
train_use_df = raw_train_use_df[raw_train_use_df['membercode'].isin(train_membercodes)].reset_index(drop=True)
# 获取train_use_df中membercode不在raw_train_use_df中数据
valid_use_df = raw_train_use_df[~raw_train_use_df['membercode'].isin(train_use_df['membercode'])]

train_wallet_df = pd.read_csv('../../../data/extracted/cust_wallet_detail_train.csv')
valid_send_df = pd.read_csv('../../../data/extracted/cust_coupon_detail_send_validation.csv')

In [97]:
train_send_df.head(10)

,marketcode,marketrulenumber,membercode,marketprovince,voucherrulecode,voucherrulename,voucherstarttime,voucherendtime,vouchertype,fulltype,usechannel,cashvalue,topamount,endnumber
0,20250805172227975,20250805004,1053006683347,53,$01-VC-17518553418576795,260减20高标号汽油券2025年爱跑节95降标客户,20230805,20230904,C02,1.0,"C05,C12",2000,26000.0,9919951
1,20250805172227975,20250805004,1053003717559,53,$01-VC-17518518998232950,6元玻璃水抵扣券-2025爱跑节-零售集字抽大奖活动-YF,20230805,20230904,C01,NaN,C01,600,NaN,9955379
2,20250212140527684,001,1042009952384,42,$01-VC-17392655640556798,Y-湖北回流-汽油220-10元满减券,20230213,20230414,C02,1.0,"C05,C12",1000,22000.0,444885
3,20250212140527684,001,1042013623236,42,$01-VC-17392660738476798,Y-湖北回流-汽油220-15元满减券,20230213,20230414,C02,1.0,"C05,C12",1500,22000.0,460504
4,20250212140527684,001,1042007051862,42,$01-VC-17392660738476798,Y-湖北回流-汽油220-15元满减券,20230213,20230414,C02,1.0,"C05,C12",1500,22000.0,485046
5,20250608212843475,20250608,1065003316754,65,$01-VC-17493891848642042,流失客户定向召回-汽油满200-20券,20230608,20230708,C02,1.0,C12,2000,20000.0,120760
6,20250608212843475,20250608,1065000778191,65,$01-VC-17493891890742042,流失客户定向召回-高品号汽油满368-40元券,20230608,20230708,C02,1.0,C12,4000,36800.0,94180
7,20240816143310731,202408080001,1010007159247,46,$01-VC-17231070584051580,石化钱包满268元减16元汽油抵扣券（睡眠客户唤醒8-9月）,20220815,20220930,C02,1.0,"C05,C12",1600,26800.0,906697
8,20250805170226507,20250805002,1053011275952,53,$01-VC-17518518845216795,260减20高标号汽油券2025年爱跑节95流失召回,20230805,20230904,C02,1.0,"C05,C12",2000,26000.0,9950603
9,20241017150301106,241092001,1014003457916,14,$01-VC-17291479051381482,石化有礼,20221017,20221116,C02,1.0,C12,1500,18800.0,9913179


In [96]:
len(set(valid_send_df['membercode'].unique())), len(set(valid_send_df['membercode'].unique()) - set(train_send_df['membercode'].unique()))

(9962, 9962)

In [92]:
len(set(train_use_df['membercode'].unique())), len(set(train_use_df['membercode'].unique()) - set(raw_train_use_df['membercode'].unique()))

(23909, 0)

In [93]:
len(set(raw_train_use_df['membercode'].unique()))

29887

In [ ]:
# 统计每个marketrulenumber的成交率
coupon_send_user_dict = {}
for idx, row in train_send_df.iterrows():
    marketrulenumber = row['marketrulenumber']
    marketcode = row['marketcode']
    coupon_id = str(marketcode) + '_' + str(marketrulenumber)
    if coupon_id not in coupon_send_user_dict:
        coupon_send_user_dict[coupon_id] = [0, 0] # send use 
    coupon_send_user_dict[coupon_id][0] += 1 
for idx, row in train_use_df.iterrows():
    marketrulenumber = row['marketrulenumber']
    marketcode = row['marketcode']
    coupon_id = str(marketcode) + '_' + str(marketrulenumber)
    if coupon_id not in coupon_send_user_dict:
        coupon_send_user_dict[coupon_id] = [0, 0] # send use 
    coupon_send_user_dict[coupon_id][1] += 1 
    coupon_send_user_dict[coupon_id][0] = max(coupon_send_user_dict[coupon_id]) 



In [42]:
sum(coupon_rate.values()) / len(coupon_rate)

0.30970425425251896

In [41]:
coupon_rate = {} 
print(len(coupon_send_user_dict))
for marketrulenumber, rate in coupon_send_user_dict.items():
    coupon_rate[marketrulenumber] = rate[1] / rate[0] if rate[0] >0 else 0 
list(coupon_rate.items())[:10]

586


[('20250805172227975_20250805004', 0.0),
 ('20250212140527684_001', 0.3984375),
 ('20250608212843475_20250608', 0.625),
 ('20240816143310731_202408080001', 0.5405405405405406),
 ('20250805170226507_20250805002', 0.045454545454545456),
 ('20241017150301106_241092001', 0.4642857142857143),
 ('20250606145907813_20250606jiang60', 0.411214953271028),
 ('20250805092521558_1', 0.011904761904761904),
 ('20250512190242557_001', 0.06746463547334058),
 ('20250512180128667_001', 0.12109375)]

In [60]:
# calc valid user coupon use rate 
user_rate = {}
for idx, row in valid_send_df.iterrows():
    marketrulenumber = row['marketrulenumber']
    marketcode = row['marketcode']
    membercode = row['membercode'] 
    coupon_id = str(marketcode) + '_' + str(marketrulenumber)
    if membercode not in user_rate:
        user_rate[membercode] = {} 
    if coupon_id not in user_rate[membercode]:
        user_rate[membercode][coupon_id] = 0.3 
    if coupon_id in coupon_rate: 
        user_rate[membercode][coupon_id] = coupon_rate[coupon_id]
user_final_rate = {} 
for k, v in user_rate.items():
    r = 1 
    for vv in v.values():
        r *= 1-vv
    r = 1-r 
    r = max(v.values()) 
    user_final_rate[k] = r  
list(user_final_rate.items())[:10]

[(1053002139364, 0.39473684210526316),
 (1014004454656, 0.7175438596491228),
 (1014006728182, 0.4642857142857143),
 (1014002254221, 0.7175438596491228),
 (1061002009946, 0.26521739130434785),
 (1035005113438, 0.7380952380952381),
 (1042013889745, 0.2690631808278867),
 (1042006255607, 0.4068627450980392),
 (1042007007215, 0.5613747954173486),
 (1023000639864, 0.37719298245614036)]

In [49]:
# output sub
valid_wallet_df = pd.read_csv('../../../data/extracted/cust_wallet_detail_validation_without_truth.csv')
valid_wallet_df.head(2)

,order_no,external_order_no,user_id,membercode,station_code,station_name,sale_time,tran_amt,receivable_amt,discounts_amt,point_amt,attributionorgcode,transactionorgcode
0,33300146PCITC2025042050109,33300146093250420112600286,4946e1ba1eb3b5c21601ff6270d472a2,1010005018581,33300146,武鸣客运加油站,2023/4/20 11:26,295.92,295.92,0.0,0.0,45.0,45
1,33300637PCITC2025042050041,33300637080250420092228374,ea9ac1b814c95442b4a10979a4476601,1045005133971,33300637,湘桂加油站,2023/4/20 9:22,148.06,148.06,0.0,0.0,45.0,45


In [51]:
# 计算 train_wallet_df中coupon_code不为Nan的比例
train_wallet_df['coupon_code'].isna().sum() / len(train_wallet_df)

0.6524742230243084

In [71]:
output = [] 
for idx, row in valid_wallet_df.iterrows():
    membercode = row['membercode']
    order_no = row['order_no']
    if membercode not in user_final_rate:
        output.append([order_no, 0.35]) 
    else:
        output.append([order_no, user_final_rate[membercode]**1.6])
output[:10]
        

[['33300146PCITC2025042050109', 0.4239550972895536],
 ['33300637PCITC2025042050041', 0.22225482269869504],
 ['33302099PCITC2025042050210', 0.32987697769322355],
 ['33300112PCITC2025042050883', 0.38012386145179144],
 ['33300463PCITC2025042050068', 0.48156093347177203],
 ['33302339PCITC2025042050033', 0.48156093347177203],
 ['33303219PCITC2025042050379', 0.4239550972895536],
 ['33301862PCITC2025042050109', 0.19442412745730833],
 ['33302338PCITC2025042050489', 0.4239550972895536],
 ['32550205PCITC2024122650345', 0.6843440908842839]]

In [72]:
sum([x[1] for x in output])/ len(output)

0.35759098904523023

In [73]:
output_df = pd.DataFrame(output, columns=['id', 'predict'])
output_df.to_csv('./submission_20250828_with_max_rate_m1.6.csv', index=False)

In [21]:
# 对send_coupon_df中的分类字段做one hot编码，准备用于sklearn模型
# 选择需要做one hot的字段
one_hot_fields = ['marketprovince', 'voucherrulecode', 'voucherrulename', 'vouchertype', 'fulltype', 'usechannel']

# 对这些字段做one hot编码
send_coupon_df_onehot = pd.get_dummies(send_coupon_df, columns=one_hot_fields, dummy_na=True)

# send_coupon_df_onehot 就是one hot后的特征表
send_coupon_df_onehot


,voucherstarttime,voucherendtime,topamount,cashvalue,marketprovince_10.0,marketprovince_11.0,marketprovince_12.0,marketprovince_13.0,marketprovince_14.0,marketprovince_15.0,...,"usechannel_C01,C16",usechannel_C02,usechannel_C05,"usechannel_C05,C12","usechannel_C05,C12,C22",usechannel_C12,usechannel_C16,"usechannel_C23,C05,C12,C22",usechannel_C30,usechannel_nan
0,20230805,20230904,26000.0,2000,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,20230805,20230904,NaN,600,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,20230213,20230414,22000.0,1000,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,20230213,20230414,22000.0,1500,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,20230213,20230414,22000.0,1500,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211707,20230607,20230708,20000.0,1800,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
211708,20230607,20230708,20000.0,1800,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
211709,20230607,20230708,20000.0,1800,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
211710,20230607,20230708,20000.0,1800,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [10]:
len(user_id)

9962

In [6]:
send_df = pd.read_csv('../../../data/extracted/cust_coupon_detail_send_validation.csv')

In [11]:
len(send_df['membercode'].unique())

9962

In [13]:
from sklearn.linear_model import LogisticRegression 

In [14]:
# 按照优惠券分类

In [75]:
len(set(valid_wallet_df['membercode'].unique()) - set(train_wallet_df['membercode'].unique()))

9962